In [73]:
import matplotlib.pyplot as plt
import numpy as np
import os
import xlrd
import pandas as pd
# import seaborn as sns
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer

# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import Binarizer
# from sklearn.tree import DecisionTreeClassifier
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

# Publich Schools Data (pubshcls.csv)

In [74]:
schools = pd.read_csv('Datasets/pubschools_info.csv')

schools = schools.loc[schools['County'] == 'San Diego']

cols_to_drop = ["NCESSchool","StreetAbr","MailStreet","MailStrAbr","MailCity",
               "MailZip","MailState","Ext","FaxNumber","Email","Phone","CharterNum",
                "FundingType","DOC","DOCType","EdOpsCode","EdOpsName","Magnet","FederalDFCDistrictID",
                "AdmFName","AdmLName","AdmEmail","YearRoundYN",'State',
                 'District', 'OpenDate',"SOC", "SOCType","GSoffered","GSserved",'ClosedDate','Virtual',"NCESDist"
               ]
#                ,"WebSite", "EILName",'Charter']

schools = schools.drop(cols_to_drop, axis=1)

clean1 = schools.loc[schools['StatusType'] == 'Active'] #Keeps only active schools
clean3 = clean1.loc[clean1['School'] != 'No Data'] #drops district records - nonschools
clean3['LastUpDate'] = pd.to_datetime(clean3['LastUpDate']) #change lastUpDate to datetime obj col
clean3 = clean3.reset_index().drop('index',axis = 1)
schools = clean3

schools = schools.replace("No Data",np.nan) #changed "No Data" into NaNs
schools['Latitude'] = schools['Latitude'].astype(float) #changed latitutde to float dtype
schools['Longitude'] = schools['Longitude'].astype(float) #changed longitude to float dtype
schools = schools.drop(['StatusType','County'],axis = 1) #dropped statustype and county col since its all active and SD county

# small_zip = schools['Zip'].str.slice(0,5).astype(float)
# schools = schools.assign(SmallZip=small_zip)

address = schools['Street'] + ',' + schools['City'] + ",CA," + schools['Zip'].str.slice(0,5)
schools = schools.assign(Address = address).drop(["Street",'City'],axis = 1)
schools = schools.assign(Zipcode = schools['Zip'].str.slice(0,5))
schools['Zipcode'] = schools['Zipcode'].astype(int)

type_of_schools = ['ELEM','HS','INTMIDJR','ELEMHIGH']
schools = schools[schools.EILCode.isin(type_of_schools)]

schools = schools.loc[schools['Charter'] == 'N']
schools = schools.drop(['EILCode','Charter'],axis = 1)

C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


* Take away Ungraded, Adult,ELEMHIGH,Pres rows
* take away Virtual col
* Keep EILName no need for code
* maybe keep website?

In [69]:

# schools[schools['EILCode'] in type_of_schools]

In [70]:
schools

,CDSCode,School,Zip,WebSite,Charter,EILCode,EILName,Latitude,Longitude,LastUpDate,Address,Zipcode
0,37103710108548,Iftin Charter,92115-3620,http://www.iftincharter.net,Y,ELEM,Elementary,32.757908,-117.07814,2020-08-13,"5465 El Cajon Boulevard,San Diego,CA,92115",92115
1,37103710115998,San Pasqual Academy,92025-5301,http://www.sanpasqualacademy.org,N,HS,High School,33.090580,-116.95009,2019-02-13,"17701 San Pasqual Valley Road,Escondido,CA,92025",92025
2,37103710120485,Davila Day,91910-3604,NaN,N,ELEM,Elementary,32.635041,-117.08832,2019-02-13,"540 G Street,Chula Vista,CA,91910",91910
3,37103710120493,Monarch,92113-1012,http://www.monarchschools.org,N,ELEMHIGH,Elementary-High Combination,32.702375,-117.15045,2020-08-24,"1625 Newton Avenue,San Diego,CA,92113",92113
4,37103710128520,San Diego County Community,92111-7399,http://www.sdcoe.net,N,ELEMHIGH,Elementary-High Combination,32.769578,-117.17984,2020-08-21,"6401 Linda Vista Road, Room 216,San Diego,CA,9...",92111
5,37103710128538,San Diego County Court,92123,http://www.sdcoe.net/jccs,N,HS,High School,32.794147,-117.15396,2019-02-13,"2801 Meadow Lark Drive,San Diego,CA,92123",92123
7,37103710136085,Scholarship Prep - Oceanside,92057-6402,http://scholarshipschools.org/,Y,ELEM,Elementary,33.232373,-117.31700,2020-08-27,"4070 Mission Avenue,Oceanside,CA,92057",92057
8,37103710137695,Community Montessori,92026-2242,http://www.cmcharter.org,Y,ELEM,Elementary,33.129426,-117.11209,2020-08-13,"1441 Montiel Road, Suite 143,Escondido,CA,92026",92026
9,37103710137752,Dimensions Collaborative,92026-2242,http://www.dimensions.org,Y,ELEMHIGH,Elementary-High Combination,33.129426,-117.11209,2020-08-13,"1441 Montiel Road, Suite 143,Escondido,CA,92026",92026
10,37103710138016,Pacific Springs Charter,91913-3953,http://www.springscharterschools.org,Y,ELEM,Elementary,32.620433,-116.97632,2020-08-13,"1615 Mater Dei Drive,Chula Vista,CA,91913",91913


In [71]:

schools

,CDSCode,School,Zip,WebSite,EILName,Latitude,Longitude,LastUpDate,Address,Zipcode
1,37103710115998,San Pasqual Academy,92025-5301,http://www.sanpasqualacademy.org,High School,33.090580,-116.95009,2019-02-13,"17701 San Pasqual Valley Road,Escondido,CA,92025",92025
2,37103710120485,Davila Day,91910-3604,NaN,Elementary,32.635041,-117.08832,2019-02-13,"540 G Street,Chula Vista,CA,91910",91910
3,37103710120493,Monarch,92113-1012,http://www.monarchschools.org,Elementary-High Combination,32.702375,-117.15045,2020-08-24,"1625 Newton Avenue,San Diego,CA,92113",92113
4,37103710128520,San Diego County Community,92111-7399,http://www.sdcoe.net,Elementary-High Combination,32.769578,-117.17984,2020-08-21,"6401 Linda Vista Road, Room 216,San Diego,CA,9...",92111
5,37103710128538,San Diego County Court,92123,http://www.sdcoe.net/jccs,High School,32.794147,-117.15396,2019-02-13,"2801 Meadow Lark Drive,San Diego,CA,92123",92123
14,37103711996545,North Coastal Consortium Schools,92069-3709,http://nccse.org,Elementary-High Combination,33.142539,-117.16504,2019-02-13,"255 Pico Avenue, Room 220,San Marcos,CA,92069",92069
15,37103716069355,San Diego County Special Education,92111-7319,http://www.sdcoe.net,Elementary-High Combination,32.769409,-117.17960,2019-11-20,"6401 Linda Vista Road, Room 409,San Diego,CA,9...",92111
17,37679676037535,Joan MacQueen Middle,91901-3129,http://www.alpineschools.net/joanmac,Intermediate/Middle/Junior High,32.825432,-116.77538,2019-02-13,"2001 Tavern Road,Alpine,CA,91901",91901
18,37679676106256,Shadow Hills Elementary,91901-2615,http://www.alpineschools.net/shadowhills,Elementary,32.831900,-116.81522,2019-02-26,"8770 Harbison Canyon Road,Alpine,CA,91901",91901
19,37679676109086,Boulder Oaks Elementary,91901-3134,http://www.alpineschools.net/boulderoaks,Elementary,32.819163,-116.77617,2020-08-13,"2320 Tavern Road,Alpine,CA,91901",91901


In [72]:
schools.to_csv(r'C:/Users/Andrew Chin/border_chlng/final_schools_table.csv')

# Misc. Stats

In [53]:
num_schools = len(schools)
unique_small_zips = schools['Zipcode'].nunique()
unique_zips = len(schools['Zip'].unique())
total_zips = len(schools['Zip'])

In [54]:
print("There are {} unique Zipcodes out of {}".format(unique_small_zips, total_zips))
print("There are {} unique Zips out of {}.".format(unique_zips,total_zips))

There are 101 unique Zipcodes out of 790
There are 703 unique Zips out of 790.


In [55]:
num_preschool = len(schools[schools['EILCode'] == 'PS'])
school_types = {}
for code in schools['EILCode'].unique():
    school_types[code] = (schools[schools['EILCode'] == code])
    
school_types['A'] #wier

,CDSCode,School,Zip,EILCode,Virtual,Latitude,Longitude,LastUpDate,Address,Zipcode
166,37681063730025,Escondido Adult,92025,A,C,33.131196,-117.08846,2020-07-16,"220 West Crest,Escondido,CA,92025",92025
191,37681303730199,Foothills Adult Education Center,92019-2015,A,C,32.797130,-116.92436,2018-08-17,"1550 Melody Lane,El Cajon,CA,92019",92019
192,37681303730249,El Cajon Adult Center,92021-6226,A,C,32.800160,-116.94594,2019-08-13,"1075 East Madison Avenue,El Cajon,CA,92021",92021
193,37681303730256,Steele Canyon Adult Center,91978-2331,A,C,32.730297,-116.92517,2018-08-17,"12440 Campo Road,Spring Valley,CA,91978",91978
201,37681303732419,Grossmont Adult,92019,A,C,32.797130,-116.92436,2020-08-25,"1550 Melody Lane,El Cajon,CA,92019",92019
301,37682963735834,Poway Adult,92064-3034,A,C,32.978786,-117.03586,2016-12-08,"13626 Twin Peaks Road,Poway,CA,92064",92064
333,37683043730900,Ramona Adult,92065-2348,A,N,33.039329,-116.86387,2020-06-26,"720 Ninth Street,Ramona,CA,92065",92065
401,37683383730280,San Diego Adult,92113,A,F,32.703991,-117.09202,2020-05-15,"4777 Imperial Avenue Room 407,San Diego,CA,92113",92113
620,37684113730132,Montgomery Adult,92154-1507,A,N,32.585313,-117.06205,2019-08-26,"3240 Palm Avenue,San Diego,CA,92154",92154
625,37684113731106,San Ysidro Adult Education Center,92173,A,N,32.558871,-117.04133,2019-08-26,"4220 Otay Mesa Road,San Ysidro,CA,92173",92173


# Important Codes

### EILCodes
* A – Adult
* ELEM – Elementary
* ELEMHIGH – Elementary-High Combination
* HS – High School
* INTMIDJR – Intermediate/Middle/Junior High
* PS – Preschool
* UG – Ungraded

### Virtual
* F = Exclusively Virtual – The school has no physical building where students meet with each other or with teachers, all instruction is virtual.
* V = Primarily Virtual – The school focuses on a systematic program of virtual instruction but includes some physical meetings among students or with teachers.
* C = Primarily Classroom – The school offers virtual courses but virtual instruction is not the primary means of instruction.
* N = Not Virtual – The school does not offer any virtual instruction.
* P = Partial Virtual – The school offers some, but not all, instruction through virtual instruction. Note: This value was retired and replaced with the Primarily Virtual and Primarily Classroom values beginning with the 2016–17 school year.

# Summary

Got rid of the columns that would not help in our project. Changed Dates to date time objects and numbers to floats. Also split the zip into a "Zipcode" which consists of only the first 5 digits. Filtered the schools to show only the active schools (not the closed nor merged schools)